In [3]:
import sys, pathlib
import pandas as pd
import numpy as np
import joblib

# Ruta al proyecto
PROJECT_ROOT = pathlib.Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg
from src import evol_utils as eu
from sklearn.preprocessing import StandardScaler

# === Paths dinámicos ===
if cfg.MODEL_TYPE == "lstm":
    MODEL_PATH = cfg.MODELS / cfg.LSTM_MODEL_NAME
    DATA_PATH  = cfg.DATA / "processed" / "lstm_data.pkl"
elif cfg.MODEL_TYPE == "lstm5d":
    MODEL_PATH = cfg.MODELS / cfg.LSTM5D_MODEL_NAME
    DATA_PATH  = cfg.DATA / "processed" / "lstm5d_data.pkl"
elif cfg.MODEL_TYPE == "gru5d":
    MODEL_PATH = cfg.MODELS / cfg.GRU5D_MODEL_NAME
    DATA_PATH = cfg.DATA / "processed" / "gru5d_data.pkl"
elif cfg.MODEL_TYPE == "xgb":
    MODEL_PATH = cfg.MODELS / cfg.XGB_MODEL_NAME
    DATA_PATH  = cfg.DATA / "processed" / "xgb_data.pkl"
elif cfg.MODEL_TYPE == "cnn5d":
    MODEL_PATH = cfg.MODELS / "cnn5d.keras"
    DATA_PATH = cfg.DATA / "processed" / "cnn5d_data.pkl"
    scaler_cnn5d = joblib.load(cfg.MODELS / "scaler_X_cnn5d.pkl")


else:
    raise ValueError(f"Modelo '{cfg.MODEL_TYPE}' no soportado")

PRICES_PATH = cfg.DATA / "raw" / "prices.parquet"

print(f"🧠 Modelo activo: {cfg.MODEL_TYPE}")

🧠 Modelo activo: lstm


In [5]:
# === Datos ===
df_prices = pd.read_parquet(PRICES_PATH).sort_index()
lstm_data = joblib.load(DATA_PATH)
tickers = lstm_data["tickers"]
df_prices = df_prices[tickers]
df_ret = np.log(df_prices / df_prices.shift(1)).dropna()

# Momentum + features
ret5 = df_ret.rolling(5).sum()
vol5 = df_ret.rolling(5).std()
momentum = (ret5 / vol5).shift(1)
df_feat = pd.concat([df_ret.shift(1), momentum], axis=1).dropna()

print("✅ df_feat shape:", df_feat.shape)

# === Cargar modelo y escalador ===
if cfg.MODEL_TYPE in ["lstm", "lstm5d","cnn5d"]:
    from tensorflow import keras
    model = keras.models.load_model(MODEL_PATH, compile=False)
elif cfg.MODEL_TYPE == "gru5d":
    from tensorflow import keras
    model = keras.models.load_model(cfg.MODELS / cfg.GRU5D_MODEL_NAME, compile=False)
    DATA_PATH = cfg.DATA / "processed" / "gru5d_data.pkl"
elif cfg.MODEL_TYPE == "xgb":
    model = joblib.load(MODEL_PATH)
else:
    raise ValueError(f"Modelo '{cfg.MODEL_TYPE}' no soportado")

# Solo para lstm5d
if cfg.MODEL_TYPE == "lstm5d":
    scaler_lstm5d = joblib.load(cfg.MODELS / "scaler_X_lstm5d.pkl")

    print(f"✅ Modelo CNN cargado y listo: {model.input_shape} → {model.output_shape}")

✅ df_feat shape: (1495, 80)


In [7]:
# === rebalanceo / main.py (fragmento relevante) ===

def rebalancear_en_fecha(fecha, df_feat, model, w_prev=None):
    try:
        idx = df_feat.index.get_loc(fecha)
        ventana = df_feat.iloc[idx - cfg.WINDOW: idx]

        if cfg.MODEL_TYPE == "lstm":
            ventana = ventana.iloc[:, :len(tickers)]
            scaler = StandardScaler()
            X_input = scaler.fit_transform(ventana.values)
            X_input = np.expand_dims(X_input, 0)
            r_hat = model.predict(X_input, verbose=0)[0]

        elif cfg.MODEL_TYPE == "lstm5d":
            X_input = scaler_lstm5d.transform(ventana.values).reshape(1, cfg.WINDOW, -1)
            r_hat = model.predict(X_input, verbose=0)[0]

        elif cfg.MODEL_TYPE == "cnn5d":
            X_input = scaler_cnn5d.transform(ventana.values).reshape(1, cfg.WINDOW, -1)
            r_hat = model.predict(X_input, verbose=0)[0]

        elif cfg.MODEL_TYPE == "gru5d":
            ventana_ret = df_ret.iloc[idx - cfg.WINDOW: idx]
            ret5 = df_ret.rolling(5).sum()
            vol5 = df_ret.rolling(5).std()
            momentum = (ret5 / (vol5 + 1e-6)).shift(1)
            ventana_mom = momentum.loc[ventana_ret.index]
            ventana = pd.concat([ventana_ret, ventana_mom], axis=1)
            scaler = StandardScaler()
            X_input = scaler.fit_transform(ventana.values).reshape(1, cfg.WINDOW, -1)
            r_hat = model.predict(X_input, verbose=0)[0]

        elif cfg.MODEL_TYPE == "xgb":
            scaler = StandardScaler()
            X_input = scaler.fit_transform(ventana.values)
            r_hat = np.array([
                model[i].predict(X_input[-1].reshape(1, -1))[0]
                for i in range(X_input.shape[1])
            ])
        else:
            raise ValueError("Tipo de modelo no reconocido")

        fecha_ret = df_feat.index[idx]
        ventana_ret = df_ret.loc[fecha_ret - pd.Timedelta(days=cfg.WINDOW*2):fecha_ret]
        Sigma = ventana_ret[-cfg.WINDOW:].cov().values

        if r_hat.shape[0] != Sigma.shape[0]:
            print(f"⚠️ Dim mismatch {fecha.date()}")
            return None

        # Optimización con penalización (solo dentro del solver)
        res = eu.resolver_optimizacion(r_hat, Sigma, w_prev=w_prev)
        w_star = eu.elegir_w_star(res, r_hat, Sigma, w_prev=w_prev)


        # Turnover medido fuera para logging
        turnover = np.sum(np.abs(w_star - w_prev)) if w_prev is not None else 1.0

        ret_bruto = df_ret.iloc[idx: idx + cfg.REBAL_FREQ].values @ w_star
        ret_neto = ret_bruto.sum() - turnover * cfg.COST_TRADE

        return {
            "fecha": fecha,
            "ret_bruto": ret_bruto.sum(),
            "ret_neto": ret_neto,
            "turnover": turnover,
            "w_star": w_star
        }

    except Exception as e:
        print(f"ERROR {fecha.date()}: {e}")
        return None


In [ ]:
# === Bucle de backtest ===

fechas = df_feat.loc[cfg.START_BACKTEST:].index
resultados = []
w_prev = np.zeros(len(tickers))  # o np.ones(...) / len(...) si quieres iniciar diversificado

for i in range(cfg.WINDOW, len(fechas) - cfg.REBAL_FREQ, cfg.REBAL_FREQ):
    fecha = fechas[i]
    out = rebalancear_en_fecha(fecha, df_feat, model, w_prev=w_prev)

    if out is not None:
        resultados.append(out)
        print(
            f"✅ {fecha.date()} | Retorno bruto {out['ret_bruto']:.4%} | "
            f"neto {out['ret_neto']:.4%} | turnover {out['turnover']:.2%}"
        )
        w_prev = out["w_star"]
    else:
        print(f"⚠️  {fecha.date()} | Resultado nulo")


ERROR 2019-04-24: NSGA-II no encontró soluciones factibles (ajusta τ o el tol de la suma).
⚠️  2019-04-24 | Resultado nulo
ERROR 2019-05-10: NSGA-II no encontró soluciones factibles (ajusta τ o el tol de la suma).
⚠️  2019-05-10 | Resultado nulo
ERROR 2019-05-30: NSGA-II no encontró soluciones factibles (ajusta τ o el tol de la suma).
⚠️  2019-05-30 | Resultado nulo
ERROR 2019-06-18: NSGA-II no encontró soluciones factibles (ajusta τ o el tol de la suma).
⚠️  2019-06-18 | Resultado nulo
ERROR 2019-07-09: NSGA-II no encontró soluciones factibles (ajusta τ o el tol de la suma).
⚠️  2019-07-09 | Resultado nulo
ERROR 2019-07-25: NSGA-II no encontró soluciones factibles (ajusta τ o el tol de la suma).
⚠️  2019-07-25 | Resultado nulo
ERROR 2019-08-13: NSGA-II no encontró soluciones factibles (ajusta τ o el tol de la suma).
⚠️  2019-08-13 | Resultado nulo
ERROR 2019-08-29: NSGA-II no encontró soluciones factibles (ajusta τ o el tol de la suma).
⚠️  2019-08-29 | Resultado nulo
ERROR 2019-09-18

In [9]:
import pandas as pd
import joblib

res_df = pd.DataFrame(resultados).set_index("fecha")
joblib.dump(res_df, cfg.RESULT / f"backtest_{cfg.MODEL_TYPE}.pkl")
print("✅ Backtest guardado:", cfg.RESULT / f"backtest_{cfg.MODEL_TYPE}.pkl")


✅ Backtest guardado: C:\Users\ferra\Documents\TFM\results\backtest_cnn5d.pkl


In [ ]:
import matplotlib.pyplot as plt

res_df["coste_acum"] = res_df["coste"].cumsum()
res_df["capital"] = (1 + res_df["retorno_net"]).cumprod()

fig, ax1 = plt.subplots(figsize=(12,6))
res_df["capital"].plot(ax=ax1, color='blue', label="Capital neto")
ax1.set_ylabel("Capital")
ax1.legend(loc="upper left")

ax2 = ax1.twinx()
res_df["coste_acum"].plot(ax=ax2, color='red', label="Coste acumulado", linestyle="--")
ax2.set_ylabel("Coste acumulado")
ax2.legend(loc="upper right")
plt.title("Capital vs Costes de Transacción Acumulados")
plt.grid(True)
plt.tight_layout()
plt.show()
